In [ ]:
import numpy as np

In [ ]:
X = np.load('X.npy')
y = np.load('y.npy')

In [ ]:
print(X.shape)
print(y.shape)

(725, 224, 224, 3)
(725,)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
X_preprocessed = (X * 2.0) - 1.0
print(f"New range: min={X_preprocessed.min()}, max={X_preprocessed.max()}")

New range: min=-1.0, max=1.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_preprocessed, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)     # Flatten the output
x = Dropout(0.2)(x)                 # Regularization to prevent overfitting
predictions = Dense(4, activation='softmax')(x) # 4 classes

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy', # Use 'sparse' if y is integers (0,1,2,3)
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=100,             # Set this high!
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stop, checkpoint] # Add the callbacks here
)

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 614ms/step - accuracy: 0.2667 - loss: 1.6857
Epoch 1: val_accuracy improved from -inf to 0.32414, saving model to best_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.2667 - loss: 1.6848 - val_accuracy: 0.3241 - val_loss: 1.4494
Epoch 2/100
17/19 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.3010 - loss: 1.5590
Epoch 2: val_accuracy improved from 0.32414 to 0.40690, saving model to best_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.3003 - loss: 1.5572 - val_accuracy: 0.4069 - val_loss: 1.3601
Epoch 3/100
17/19 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3498 - loss: 1.4282
Epoch 3: val_accuracy improved from 0.40690 to 0.48966, saving model to best_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.3502 - loss: 1.4271 - val_accuracy: 0.4897 - val_loss: 1.2822
Epoch 4/100
17/19 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3907 - loss: 1.3815
Epoch 4: val_accuracy did not improve from 0

In [ ]:
# 1. Unfreeze the base model
base_model.trainable = True

# 2. Refreeze the early layers (Optional but recommended for small data)
# MobileNetV2 has 155 layers. Let's tune the last ~50 layers.
print(f"Number of layers in the base model: {len(base_model.layers)}")

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# 3. Recompile with a VERY LOW learning rate
# This is the most important step!
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # 10x smaller than before
    metrics=['accuracy']
)

print("\nStarting Fine-Tuning...")

# 4. Continue training
# We train for fewer epochs usually, as it converges quickly
total_epochs = 100 + 50 # Previous epochs + 50 new ones

history_fine = model.fit(
    X_train, y_train,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1], # Start from where we left off
    validation_data=(X_test, y_test),
    callbacks=[early_stop] # Use the same early stopping
)

Number of layers in the base model: 154

Starting Fine-Tuning...
Epoch 100/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.8348 - loss: 0.4863 - val_accuracy: 0.8690 - val_loss: 0.3747
Epoch 101/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.8713 - loss: 0.4002 - val_accuracy: 0.8690 - val_loss: 0.3640
Epoch 102/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9245 - loss: 0.3169 - val_accuracy: 0.8690 - val_loss: 0.3556
Epoch 103/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.9275 - loss: 0.3102 - val_accuracy: 0.8690 - val_loss: 0.3441
Epoch 104/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9597 - loss: 0.2179 - val_accuracy: 0.8690 - val_loss: 0.3335
Epoch 105/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.9568 - loss: 0.2221 - val_accuracy: 0.8690 - val_loss: 0.3222
Epoch 106/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.9670 - loss: 0.1741 - val_accuracy: 0.8690 - val_loss: 0.3133
Epoch 107/150
19/19 ━━━━━━

In [ ]:
# Save the final high-accuracy model
model.save('mobilenet_v2_finetuned_95acc.keras')
print("Model saved successfully.")

Model saved successfully.
